In [7]:
import pandas as pb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import os.path
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, KFold
import gzip

from saxpy.znorm import znorm
from saxpy.sax import ts_to_string
from saxpy.alphabet import cuts_for_asize

In [8]:
X = []
Y = []

for root, dirs, files in os.walk('Recording Data'):
    for file in files:
        if file.endswith(".txt") and file != 'Subject codes.txt':
            file_path = os.path.join(root, file)
            match root[-1]:
                case '1' | '2':
                    has_dyslexia = 1
                case _:
                    has_dyslexia = 0
            data = pd.read_csv(file_path, sep='\t', index_col=None, decimal=",")
            data['r_dist'] = (data['RX'].diff() ** 2 + data['RY'].diff() ** 2) ** 0.5
            data['l_dist'] = (data['LX'].diff() ** 2 + data['LY'].diff() ** 2) ** 0.5
            added_data = np.array([list(ts_to_string(znorm(np.array(x)), cuts_for_asize(8))) for x in list(zip(data['r_dist'][1:], data['l_dist'][1:]))])
            X.append(added_data)
            Y.append(has_dyslexia)

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [10]:
k_m = [1, 5, 10, 15, 20, 25]

In [13]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)

f_scores = {}
for k in k_m:
    f_scores[k] = [[], []]

for k in k_m:
    for train_index, test_index in kf.split(X):
        X_train, X_test = [X[i] for i in train_index], [X[i] for i in test_index]
        Y_train, Y_test = [Y[i] for i in train_index], [Y[i] for i in test_index]
        predicts = []
        for x1 in X_test:
            x1 = np.array(x1)
            Cx1 = len(gzip.compress(x1))
            distance_from_x1 = []
            for x2 in X_train:
                x2 = np.array(x2)
                Cx2 = len(gzip.compress(x2))
                x1x2 = np.concatenate((x1, x2))
                Cx1x2 = len(gzip.compress(np.array(x1x2)))
                ncd = (Cx1x2 - min(Cx1 , Cx2)) / max (Cx1 , Cx2 )
                distance_from_x1.append(ncd)
                        
            sorted_idx = np.argsort(np.array(distance_from_x1))
            top_k_class = np.array(Y_train)[sorted_idx[:k]].tolist()
            predict_class = max(set(top_k_class), key = top_k_class.count)
            predicts.append(predict_class)

        sensitivity, specificity = calculate_sensitivity_specificity(Y_test, predicts)
        f1 = classification_report(Y_test, predicts, output_dict=True)['macro avg']['f1-score']
        f_scores[k][0].append(f1)
        f_scores[k][1].append([sensitivity, specificity])

In [14]:
sensitivity = {k: sum([scores[0] for scores in f_scores[k][1]]) / len([scores[0] for scores in f_scores[k][1]]) for k in k_m}
specificity = {k: sum([scores[1] for scores in f_scores[k][1]]) / len([scores[1] for scores in f_scores[k][1]]) for k in k_m}
data = [f_scores[k][0] + [sum(f_scores[k][0]) / len(f_scores[k][0]), sensitivity[k], specificity[k]] for k in k_m]

In [15]:
table = pd.DataFrame(data, index=k_m, columns=['1_split', '2_split', '3_split', '4_split', '5_split', 'avg_score', 'sensitivity', 'specificity'])

In [16]:
table

,1_split,2_split,3_split,4_split,5_split,avg_score,sensitivity,specificity
1,0.836765,0.881410,0.891813,0.756044,0.836765,0.840559,0.831373,0.850067
5,0.945906,1.000000,0.918919,0.863267,0.918919,0.929402,0.957778,0.902601
10,0.945906,0.969796,0.837719,0.863267,0.945906,0.912519,0.935556,0.887216
15,0.945906,0.969796,0.837719,0.861215,0.918919,0.906711,0.945556,0.864925
20,0.945906,0.969796,0.837719,0.863267,0.918919,0.907121,0.935556,0.876690
25,0.945906,0.969796,0.837719,0.863267,0.891813,0.901700,0.924444,0.876690


In [11]:
def calculate_sensitivity_specificity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    
    tn, fp, fn, tp = cm.ravel()
    
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    return sensitivity, specificity